In [ ]:
"""
改进id3_learn.ipynb中的不足,在python的pandas和numpy中找寻相关的api简化一些函数。
"""

In [ ]:
"""
计算初始熵值的改进方法，pandas实现。耗时耗力。
"""

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dataset = np.array([(0,2,0,0,0), (0,2,0,1,0), (1,2,0,0,1), (2,1,0,0,1), 
                   (2,0,1,0,1), (2,0,1,1,0), (1,0,1,1,1), (0,1,0,0,0), 
                   (0,0,1,0,1), (2,1,1,0,1), (0,1,1,1,1), (1,1,0,1,1), 
                   (1,2,1,0,1), (2,1,0,1,0)])

In [13]:
#dataset[:,0]

array([0, 0, 1, 2, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2])

In [12]:
#按列循环
k = []
for i in range(0,5):
    k.append(dataset[:,i])
k

[array([0, 0, 1, 2, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2]),
 array([2, 2, 2, 1, 0, 0, 0, 1, 0, 1, 1, 1, 2, 1]),
 array([0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]),
 array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1]),
 array([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])]

In [16]:
value = 1
[m, n] = dataset.shape
retDataSet = []
for j in dataset[:,0]:
    if j == value:
        retDataSet.append(j)
retDataSet

[1, 1, 1, 1]

In [23]:
#求熵解决方案,完美解决问题
[m,n] = dataset.shape
label = dataset[:,n-1]
label_del = np.unique(label)
entropy = 0
for i in label_del:
    condition = (label == i)
    a = np.extract(condition, label)
    b = len(a)/len(label)
    entropy = entropy - b*math.log(b,2)

In [3]:
#封装上述函数,参数是一个numpy的一维数组,这里考虑一下是否可以传入一维数组无法做到看来要修改
def calEntropy(label):
    label_del = np.unique(label)
    entropy = 0
    for i in label_del:
        condition = (label == i)
        a = np.extract(condition, label)
        b = len(a)/len(label)
        entropy = entropy - b*math.log(b,2)
    return entropy

In [14]:
#找最佳特征,算出所有特征的信息熵
[m,n] = dataset.shape
newentropy = []
for j in range(0,n-1):#对所有的列进行循环
    k = np.unique(dataset[:,j])
    newentropytemp = 0
    for i in k:
        condition = (dataset[:,j] == i)
        a = np.extract(condition, dataset[:,n-1])#a是一个array对象，是满足第j列各属性值时最后一列
        prob = len(a)/m
        newentropytemp = newentropytemp + prob*calEntropy(a)#需要定义求解一个一维数组的熵值
    newentropy.append(newentropytemp)

#计算信息增益最大的情况
basentropy = calEntropy(dataset[:,n-1])#数字和列表之间不能计算，把newentropy转化为numpy数组
newentropy = np.array(newentropy)#下面数组和数字之间可以进行计算。
infoGain = basentropy - newentropy

#求得最佳分裂特征的列编号
T = np.argmax(infoGain) + 1 #infoGain是一维数组，argmax的返回值才是最大值的索引

In [4]:
#封装上述函数(找最佳分裂特征)
def choseBsetF(dataset):
    [m,n] = dataset.shape
    newentropy = []
    for j in range(0,n-1):#对所有的列进行循环
        k = np.unique(dataset[:,j])
        newentropytemp = 0
        for i in k:
            condition = (dataset[:,j] == i)
            a = np.extract(condition, dataset[:,n-1])#a是一个array对象，是满足第j列各属性值时最后一列
            prob = len(a)/m
            newentropytemp = newentropytemp + prob*calEntropy(a)#需要定义求解一个一维数组的熵值
        newentropy.append(newentropytemp)

    #计算信息增益最大的情况
    basentropy = calEntropy(dataset[:,n-1])#数字和列表之间不能计算，把newentropy转化为numpy数组
    newentropy = np.array(newentropy)#下面数组和数字之间可以进行计算。
    infoGain = basentropy - newentropy

    #求得最佳分裂特征的列编号
    T = np.argmax(infoGain) + 1 #infoGain是一维数组，argmax的返回值才是最大值的索引
    
    return T#返回值是要选择的列

In [42]:
#根据最佳特征分裂数据集，讨论T = 1的情况，列1为最佳的分裂属性特征时
#numpy有相关的选择数据集的方法吗？研究numpy的split函数，不能满足要求。
#输入值是所选择的列的值，输出集合吗？至少会输出两个以上的集合。
#对这些集合又要进行选择最佳特征。递归太难了。
label = dataset[:,0]
label_num = np.unique(label)
kd = []
for i in label_num:
    condition = (label == i)
    p = []
    for k in range(0,5):
        j = np.extract(condition, dataset[:,k])
        p.append(j)
    p = np.array(p)
    kd.append(p.T)
kd#包含三个数组的列表，这里应该是处理完了

[array([[0, 2, 0, 0, 0],
        [0, 2, 0, 1, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1],
        [0, 1, 1, 1, 1]]), array([[1, 2, 0, 0, 1],
        [1, 0, 1, 1, 1],
        [1, 1, 0, 1, 1],
        [1, 2, 1, 0, 1]]), array([[2, 1, 0, 0, 1],
        [2, 0, 1, 0, 1],
        [2, 0, 1, 1, 0],
        [2, 1, 1, 0, 1],
        [2, 1, 0, 1, 0]])]

In [57]:
#封装，根据最佳属性分裂数据,返回含有所有子集的列表
def splitData(dataset,t):
    [m,n] = dataset.shape
    label = dataset[:,t-1]
    label_num = np.unique(label)
    kd = []
    for i in label_num:
        condition = (label == i)
        p = []
        for k in range(0,n):
            j = np.extract(condition, dataset[:,k])
            p.append(j)
        p.pop(t-1)
        p = np.array(p)
        kd.append(p.T)
    return kd

In [ ]:
#函数递归，可以写递归函数了。
